<a href="https://colab.research.google.com/github/tryambakgour/Coursera_Capstone/blob/master/Week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests

In [0]:
apartment_list = []
for p in range(1,35):
    url = 'https://www.apartmentlist.com/tx/houston/page-{}'.format(p)
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
    for item in soup.find_all('div', class_='css-dfx0e1 e1k7pw6k0'):
        apartment_list.append([item.a.text, item.find('div', class_='css-17xjl8p e1k7pw6k5').text])

In [0]:
import pandas as pd

In [8]:
df_apartment = pd.DataFrame(apartment_list, columns=['Apartment Name', 'Address'])
df_apartment.head(10)

,Apartment Name,Address
0,Camden Highland Village,"3255 Las Palmas St, Houston, TX"
1,Asheville at Spring Branch,"10800 Clay Rd, Houston, TX"
2,Brisa at Shadowlake,"2840 Shadowbriar Dr, Houston, TX"
3,City Place,"306 McGowen St, Houston, TX"
4,Memorial,"600 Nottingham Oaks Trl, Houston, TX"
5,1711 Caroline Apartments,"1711 Caroline Ave, Houston, TX"
6,Houston House,"1617 Fannin St, Houston, TX"
7,Rice Lofts,"909 Texas Ave, Houston, TX"
8,Camden Travis Street,"2700 Travis St, Houston, TX"
9,Camden Woodson Park,"14633 Woodson Park Dr, Houston, TX"


In [0]:
df_apartment = df_apartment[~df_apartment.duplicated()]

In [10]:
df_apartment[df_apartment.duplicated(['Address'], keep=False)]

,Apartment Name,Address
110,Tuscany Court Apartments,"1901 Augusta Dr, Houston, TX"
236,Tuscany Apartments,"1100 Bering Dr, Houston, TX"
238,Tuscany Gate Apartments,"1100 Bering Dr, Houston, TX"
717,Tuscany Oaks Apartments,"1901 Augusta Dr, Houston, TX"
754,Tuscany Villas Apartments,"1100 Bering Dr, Houston, TX"


In [0]:
df_apartment.loc[205, 'Address'] = '2525 Augusta Dr, Houston, TX'
df_apartment.loc[209, 'Address'] = '1801 Bering Dr, Houston, TX'
df_apartment.loc[463, 'Address'] = '824 Bering Dr, Houston, TX'

In [12]:
print(df_apartment['Apartment Name'].unique().shape, df_apartment['Address'].unique().shape)

(738,) (735,)


In [0]:
df_apartment.to_csv('Houston Apartment List', index=False)

In [0]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [16]:
geolocator = Nominatim(user_agent="Apartment_explorer")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)



RateLimiter caught an error, retrying (0/2 tries). Called with (*('770 N Eldridge Pkwy, Houston, TX',), **{}).
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/geopy/geocoders/base.py", line 344, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/usr/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/usr/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.6/urllib/request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/lib/python3.6/urllib/request.py", line 1321, in do_open
    r = h.getresponse()
  File "/usr/lib/python3.6/http/client.py", line 1346, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 307, in begin
    version, status, reason = self

KeyboardInterrupt: ignored

In [33]:
Location_Object_Series = df_apartment['Address'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('770 N Eldridge Pkwy, Houston, TX',), **{}).
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/geopy/geocoders/base.py", line 344, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/usr/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/usr/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.6/urllib/request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/lib/python3.6/urllib/request.py", line 1321, in do_open
    r = h.getresponse()
  File "/usr/lib/python3.6/http/client.py", line 1346, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 307, in begin
    version, status, reason = self

KeyboardInterrupt: ignored

In [17]:
Location_Object_Series.isna().sum()

NameError: ignored

In [18]:
Location_Object_Series[Location_Object_Series.isna()]

NameError: ignored

In [19]:
df_apartment['Latitude'] = Location_Object_Series.apply(lambda x: x.latitude if x else None)
df_apartment['Longitude'] = Location_Object_Series.apply(lambda x: x.longitude if x else None)
df_apartment.head()

NameError: ignored

In [20]:
df_apartment[df_apartment['Latitude'].isna()]

KeyError: ignored

In [21]:
df_apartment = df_apartment[~((df_apartment['Latitude']>30.131527)|(df_apartment['Longitude']>-95.044849))]

KeyError: ignored

In [0]:
df_apartment = df_apartment.dropna(axis=0).reset_index(drop=True)

In [24]:
df_apartment = df_apartment[~(df_apartment['Longitude']<-95.794025)]

KeyError: ignored

In [23]:
df_apartment.shape

(738, 2)

In [22]:
address = 'Houston, TX, USA'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Houston are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Houston are 29.7589382, -95.3676974.


In [0]:
import folium

In [27]:
houston_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for name, lat, lng in zip(df_apartment['Apartment Name'], df_apartment['Latitude'], df_apartment['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        popup=name,
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(houston_map)

houston_map

KeyError: ignored

In [0]:
CLIENT_ID = 'FJ5QPYD3ITN2VXVGUU32WRMNYWFDX0HVA0TRT5CKV23ZPBFZ'
CLIENT_SECRET = '1KZWHCCMXXYEFOU5QEJHPX1EDL1XA421VFLOAOIED25F3I0E'
VERSION = 20200503
RADIUS = 750
LIMIT = 100

In [30]:
venue_list = []
for name, lat, lng in zip(df_apartment['Apartment Name'], df_apartment['Latitude'], df_apartment['Longitude']):

    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, RADIUS, LIMIT)
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    for v in results:
        venue_list.append([name, v['venue']['name'], v['venue']['categories'][0]['name'],
                           v['venue']['location']['lat'], v['venue']['location']['lng']])

df_venues = pd.DataFrame(venue_list)
df_venues.columns = ['Apartment', 'Venue', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
df_venues.head()

KeyError: ignored

In [31]:
df_venues.shape

NameError: ignored

In [32]:
venue_count.describe()

NameError: ignored

In [0]:
from matplotlib import pyplot as plt

In [35]:
BinEdge = np.linspace(0, 100, 11)
plt.hist(venue_count['Venue Count'], bins=BinEdge, color='#7993ab')

plt.xticks(BinEdge)
plt.xlabel('Number of Venues')
plt.ylabel('Count')

plt.savefig('bar chart', dpi=300)
plt.show()

NameError: ignored

In [36]:

df_apartment = df_apartment.join(venue_count, on='Apartment Name')
df_apartment.head()

NameError: ignored

In [37]:
df_apartment['Venue Count'].isna().any()

KeyError: ignored

In [0]:
from matplotlib import cm
from matplotlib import colors

In [39]:
colors_array = cm.coolwarm(df_apartment['Venue Count']/100)
colors_list = [colors.rgb2hex(i) for i in colors_array]

KeyError: ignored

In [40]:
houston_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for name, lat, lng, c in zip(df_apartment['Apartment Name'], df_apartment['Latitude'], df_apartment['Longitude'], colors_list):
    folium.CircleMarker(
        [lat, lng],
        popup=name,
        radius=3,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.7).add_to(houston_map)

houston_map

KeyError: ignored

In [41]:
df_venue_dummies = pd.get_dummies(df_venues['Venue Category'])
df_venue_dummies.insert(0, 'Apartment Name', df_venues['Apartment'])
df_venue_dummies.head()

NameError: ignored

In [42]:
df_dummy_grouped = df_venue_dummies.groupby('Apartment Name').mean()
df_dummy_grouped.head()

NameError: ignored

In [43]:
op10_list=[]
for ind, row in df_dummy_grouped.iterrows():
    row_sorted = row.sort_values(ascending=False)
    row_sorted = row_sorted[row_sorted!=0]
    if len(row_sorted)>=10:
        top10_list.append([ind] + list(row_sorted[:10].index))
    else:
        top10_list.append([ind] + list(row_sorted.index))
        
df_top10 = pd.DataFrame(top10_list)

col_names = ['Apartment Name']
temp = ['st', 'nd', 'rd'] + ['th']*7
for i in range(10):
    col_names.append('{}{} Most Common Venue'.format(i+1, temp[i]))
    
df_top10.columns = col_names
df_top10.head()

NameError: ignored

In [0]:
from sklearn.cluster import KMeans

In [45]:
k = 5
k_means = KMeans(n_clusters=k, n_init=12)
k_means.fit(df_dummy_grouped)
k_means.labels_

NameError: ignored

In [46]:
df_top10.insert(1, 'Cluster Label', k_means.labels_)
df_top10.head()

NameError: ignored

In [47]:
df_combined = df_apartment.join(df_top10.set_index('Apartment Name'), on='Apartment Name')
df_combined.head()

NameError: ignored

In [48]:
colors_array_2 = cm.rainbow(np.linspace(0, 1, k))
colors_list_2 = [colors.rgb2hex(i) for i in colors_array_2]

NameError: ignored

In [49]:
houston_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for apt, label, lat, lng in zip(df_combined['Apartment Name'], df_combined['Cluster Label'],
                          df_combined['Latitude'], df_combined['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        popup=apt+' Cluster '+str(label),
        radius=3,
        color=colors_list_2[label],
        fill=True,
        fill_color=colors_list_2[label],
        fill_opacity=0.7).add_to(houston_map)

houston_map

NameError: ignored

In [50]:
df_combined.loc[df_combined['Cluster Label']==4].iloc[:, [0]+list(range(6,16))]

NameError: ignored